# Classical ML Training and Results V2 

## INITIALIZATION

In [5]:
from sklearn import svm
from sklearn.model_selection import ShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle 
import statistics as stats

## MODEL INITIALIZATION

In [3]:
# read data from csv file and put into one matrix 
# same process of v1
def readData(filename,x1):
    f = open("data/"+filename,'r')
    for row in f:
        data = [int(p) for p in row.split(',')]
        x1.append(np.array(data))
    return 

x = []
readData("getOutBed.csv",x)
readData("getOnBed.csv",x)
readData("layOnBed.csv",x)
readData("sitOnBed.csv",x)

In [4]:
# make X an numpy array and initialize Y 
x = np.array(x)
y = np.array([1]*120+[0]*360)

## FEATURE EXTRACTION

In [8]:
x_feature = []
for r in x:
    s1, s2, s3 = r[:20000], r[20000:40000], r[40000:]
    temp1 = [stats.mean(s1), stats.median(s1), stats.variance(s1), max(s1), min(s1)]
    temp2 = [stats.mean(s2), stats.median(s2), stats.variance(s2), max(s2), min(s2)]
    temp3 = [stats.mean(s3), stats.median(s3), stats.variance(s3), max(s3), min(s3)]
#     print temp1
    x_feature.append(np.array(temp1+temp2+temp3))
